In [1]:
import sys
import pandas as pd
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm
from skimage.feature import local_binary_pattern
import os

sys.path.append(os.path.join(os.path.dirname(os.getcwd()),"modules"))

In [2]:
import NbrRegionSegment as seg
from MarinePredator.MPA import MPA

In [3]:
def get_fit_oneimg(name, wr, wg, wc, thres):
    #read the image
    img = cv.imread("../Dataset/" + name + ".jpg")

    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    #LBP part
    radius = 1
    points = 8*radius 

    lbp = local_binary_pattern(gray_img, points, radius, "uniform").astype("uint8")

    #segment image
    r = seg.RegionExtract(img, lbp, thres, wr, wg, wc)
    r1, r2 = r

    #find the fitness level
    intra, inter = seg.GetAllSegVariance(gray_img, r1, r2.shape[0])

    return intra+inter

In [4]:
#fitness function to be used in the marine predator algo
def fitness_func(wr, wg, wc, thres):
    #read the csv file
    data = pd.read_csv("../CSV/reduced.csv")

    #fitness for each image against each predator
    fit = list()

    #fetch the image file
    for entry in tqdm(data.values):
        #push the fitness into the list
        fit.append(get_fit_oneimg(entry[2], wr, wg, wc, thres)) #-> fitness is the sum of the inter and intra variance and is a decreasing function
    
    #return the fitness as the avg of all fitness
    return np.mean(fit)

In [5]:
# fitness_func(.4, .4, .8, 25)

In [6]:
#algo params for the MPA
max_iter = 30
num_agents = 15

#wr, wg, wc, thres
lb = [0, 0, 0, 10]
ub = [0.99999999, 0.999999999, 0.99999999, 50]
num_var = 4

#create a class object for the metaheuristic algorithm
mpa = MPA(num_agents, max_iter, lb, ub, num_var, fitness_func)

In [7]:
mpa.initialize()

In [8]:
mpa.Prey.shape

(15, 4)

In [9]:
mpa.Prey

array([[1.85668820e-01, 6.38801581e-01, 2.23865347e-01, 2.10296513e+01],
       [7.78548385e-01, 7.29329505e-01, 1.40662736e-01, 2.67932709e+01],
       [9.27651360e-02, 2.31024249e-01, 3.97487058e-03, 2.04443801e+01],
       [2.64839294e-01, 7.47615211e-02, 9.87901551e-01, 1.23409268e+01],
       [4.42479031e-01, 1.56627975e-01, 5.73565848e-01, 4.17396313e+01],
       [9.12758541e-01, 6.20504215e-01, 2.10417041e-02, 2.40394980e+01],
       [9.74400375e-01, 8.20195425e-01, 9.67596226e-01, 4.83470349e+01],
       [9.68124729e-01, 8.17370616e-01, 6.61355947e-01, 3.17179365e+01],
       [9.45241630e-01, 6.39281888e-02, 6.58706849e-01, 3.99612412e+01],
       [4.40157328e-01, 8.19150033e-01, 4.81929599e-02, 1.90617749e+01],
       [2.77449820e-01, 4.10193072e-01, 5.12627245e-01, 1.18523807e+01],
       [9.28937317e-01, 9.50493257e-01, 8.69258268e-01, 3.45286374e+01],
       [9.86232095e-01, 8.93989490e-02, 5.06105533e-01, 1.19834118e+01],
       [4.48457575e-01, 2.15945521e-01, 4.32854722e

In [10]:
import sys
import pandas as pd
import cv2 as cv
import numpy as np
from tqdm import tqdm
from skimage.feature import local_binary_pattern
import os
import multiprocessing as mp

import time

sys.path.append(os.path.join(os.getcwd(),"modules"))


# In[2]:

import NbrRegionSegment as seg
from MarinePredator.MPA import MPA

# In[17]:


def get_fit_oneimg(name, wr, wg, wc, thres, res):
    #read the image
    img = cv.imread("Dataset/" + name + ".jpg")
    img = cv.resize(img, (0,0), fx = 0.5, fy = 0.5)

    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    #LBP part
    radius = 1
    points = 8*radius 

    lbp = local_binary_pattern(gray_img, points, radius, "uniform").astype("uint8")

    #segment image
    r = seg._RegionExtractWrapper(img, lbp, thres, wr, wg, wc)
    r1, r2 = r

    #find the fitness level
    intra, inter = seg._GetAllVariance(gray_img, r1, r2.shape[0])

    res.put(intra+inter)


# In[18]:


#fitness function to be used in the marine predator algo
def fitness_func(wr, wg, wc, thres):
    #read the csv file
    data = pd.read_csv("CSV/reduced.csv")

    #fitness for each image against each predator
    # ctx = mp.set_start_method("forkserver")
    fit = mp.Queue()

    p = [mp.Process(target = get_fit_oneimg, args=(entry[2], wr, wg, wc, thres, fit)) for entry in data.values]

    for proc in p:
        proc.start()

    for proc in p:
        proc.join()

    #return the fitness as the avg of all fitness
    arr = [fit.get() for i in range(len(data.values))]
    return np.mean(arr)

#%%

# t = time.time()
# fitness_func(0.4, .4, .8, 25)
# print(time.time()-t)


#algo params for the MPA
max_iter = 30
num_agents = 15

#wr, wg, wc, thres
lb = [0, 0, 0, 10]
ub = [0.99999999, 0.999999999, 0.99999999, 50]
num_var = 4

#create a class object for the metaheuristic algorithm
mpa = MPA(num_agents, max_iter, lb, ub, num_var, fitness_func)

mpa.initialize()

for i in tqdm(mpa.iter_gen(), total = mpa.maxItr):
    pass


# # In[ ]:






  0%|          | 0/30 [00:00<?, ?it/s]
Iteration 0


FileNotFoundError: [Errno 2] No such file or directory: 'Project_2/CSV/reduced.csv'